In [ ]:
# !pip install openai
# !pip install python-dotenv
# !pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.7 MB/s eta 0:00:00


In [8]:

import os
import openai

from langchain import OpenAI, SQLDatabase, SQLDatabaseChain
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

dotenv_path = '/content/drive/MyDrive/Colab Notebooks/Library/colab.env'
load_dotenv(dotenv_path)

db = SQLDatabase.from_uri(os.environ['POSTGRES_URI'])
llm = OpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],temperature=0)



In [9]:
## Setting up a chain to hook up the database with the LLM
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

In [10]:
db_chain.run("How many customers are there?")



> Entering new SQLDatabaseChain chain...
How many customers are there?
SQLQuery:SELECT COUNT(*) FROM customers;
SQLResult: [(20,)]
Answer:There are 20 customers.
> Finished chain.


'There are 20 customers.'

In [12]:
db_chain.run("Which customers pay the highest premium for policies?")



> Entering new SQLDatabaseChain chain...
Which customers pay the highest premium for policies, when looking at all products that they purchased?
SQLQuery:SELECT customer_name, MAX(policy_premium) 
FROM customers 
INNER JOIN policy 
ON customers.customer_id = policy.customer_id 
GROUP BY customer_name 
ORDER BY MAX(policy_premium) DESC 
LIMIT 5;
SQLResult: [('Sarah Wilson', Decimal('1190.10')), ('Liam Scott', Decimal('1160.20')), ('Mia Hall', Decimal('1150.20')), ('David Thompson', Decimal('1150.10')), ('Joseph Walker', Decimal('1130.70'))]
Answer:The customers who pay the highest premium for policies are Sarah Wilson, Liam Scott, Mia Hall, David Thompson, and Joseph Walker.
> Finished chain.


'The customers who pay the highest premium for policies are Sarah Wilson, Liam Scott, Mia Hall, David Thompson, and Joseph Walker.'

In [13]:
db_chain.run("Which customers purchased the most number of products?")



> Entering new SQLDatabaseChain chain...
Which customers purchased the most number of products?
SQLQuery:SELECT customer_name, COUNT(*) AS num_products 
FROM customers 
INNER JOIN policy ON customers.customer_id = policy.customer_id 
GROUP BY customer_name 
ORDER BY num_products DESC 
LIMIT 5;
SQLResult: [('Joseph Walker', 4), ('Bob Johnson', 4), ('David Thompson', 4), ('Liam Scott', 4), ('Emma Allen', 4)]
Answer:The customers who purchased the most number of products are Joseph Walker, Bob Johnson, David Thompson, Liam Scott, and Emma Allen.
> Finished chain.


'The customers who purchased the most number of products are Joseph Walker, Bob Johnson, David Thompson, Liam Scott, and Emma Allen.'

In [14]:
db_chain.run("When is Bob Johnson's next premium due?")



> Entering new SQLDatabaseChain chain...
When is Bob Johnson's next premium due?
SQLQuery:SELECT next_payment_due FROM policy WHERE customer_id = (SELECT customer_id FROM customers WHERE customer_name = 'Bob Johnson');
SQLResult: [(datetime.date(2022, 4, 10),), (datetime.date(2022, 2, 26),), (datetime.date(2022, 8, 14),), (datetime.date(2022, 2, 27),)]
Answer:Bob Johnson's next premium is due on April 10th, 2022.
> Finished chain.


"Bob Johnson's next premium is due on April 10th, 2022."

In [17]:
db_chain.run("Which products did Liam Scott purchase?")



> Entering new SQLDatabaseChain chain...
Which products did Liam Scott purchase?
SQLQuery:SELECT product_name FROM products 
		  INNER JOIN policy ON products.product_id = policy.product_id 
		  INNER JOIN customers ON policy.customer_id = customers.customer_id 
		  WHERE customers.customer_name = 'Liam Scott'
		  LIMIT 5;
SQLResult: [('Indexed Universal Life',), ('Variable Deferred Annuity',), ('Indexed Universal Life',), ('Variable Deferred Annuity',)]
Answer:Liam Scott purchased Indexed Universal Life and Variable Deferred Annuity.
> Finished chain.


'Liam Scott purchased Indexed Universal Life and Variable Deferred Annuity.'

In [15]:
db_chain.run("When looking at premium for all products, which customers pay the most premium?")



> Entering new SQLDatabaseChain chain...
When looking at premium for all products, which customers pay the most premium?
SQLQuery:SELECT customer_name, SUM(policy_premium) AS total_premium FROM customers c INNER JOIN policy p ON c.customer_id = p.customer_id GROUP BY customer_name ORDER BY total_premium DESC LIMIT 5;
SQLResult: [('Liam Scott', Decimal('4051.00')), ('Joseph Walker', Decimal('3773.10')), ('David Thompson', Decimal('3362.40')), ('Jane Smith', Decimal('3300.90')), ('Sarah Wilson', Decimal('2950.40'))]
Answer:The customers who pay the most premium are Liam Scott, Joseph Walker, David Thompson, Jane Smith, and Sarah Wilson.
> Finished chain.


'The customers who pay the most premium are Liam Scott, Joseph Walker, David Thompson, Jane Smith, and Sarah Wilson.'

In [ ]:
## use OpenAI's gpt-3.5-turbo model and the chat completions endpoint.

# This helper function will make it easier to use prompts and look at the generated outputs:

# openai.api_key = os.environ['OPENAI_API_KEY']

# llm2 = ChatOpenAI(temperature=0)
# def get_completion(prompt, model="gpt-3.5-turbo"):
#     messages = [{"role": "user", "content": prompt}]
#     response = openai.ChatCompletion.create(
#         model=model,
#         messages=messages,
#         temperature=0, # this is the degree of randomness of the model's output
#     )
#     return response.choices[0].message["content"]

# get_completion("Hi")

'Hello! How can I assist you today?'